# Data Cleaning

## Import needed libraries

In [1]:
import pandas as pd
import numpy as np
import pprint

# For converting past dollars to 2018 dollars
# https://github.com/datadesk/cpi
import cpi
cpi.update()

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

pd.options.display.max_rows = 400
pd.options.display.max_columns = 50

## Import Box Office Mojo dataset and fix small issues

We read in the Box Office Mojo.

We remove redundant columns and rename the labels with `_mojo` suffixes to avoid confusion. 

We skip the columns `title` and `release_year` though, since we will be joining on those columns.

In [3]:
mojo = pd.read_csv("movie_data.csv", parse_dates=["release_date_formatted"])

mojo.drop(labels=[
    'runtime',
    'release_date',
    'domestic_gross',
    'foreign_gross',
    'worldwide_gross',
    'adjusted_domestic_gross_2019'
], axis=1, inplace=True)

mojo.rename(mapper={
    'distributor': 'distributor_mojo',
    'rating': 'rating_mojo',
    'genres': 'genres_mojo',
    'production_budget': 'production_budget_mojo',
    'runtime_formatted': 'runtime_mojo',
    'release_date_formatted': 'release_date_mojo',
    'domestic_gross_formatted': 'domestic_gross_mojo',
    'foreign_gross_formatted': 'foreign_gross_mojo',
    'worldwide_gross_formatted': 'worldwide_gross_mojo',
    'adjusted_domestic_gross_2019_formatted': 'adjusted_domestic_gross_2019_mojo',
    'director1': 'director1_mojo',
    'director2': 'director2_mojo',
    'writer1': 'writer1_mojo',
    'writer2': 'writer2_mojo',
    'writer3': 'writer3_mojo',
    'actor1': 'actor1_mojo',
    'actor2': 'actor2_mojo',
    'actor3': 'actor3_mojo',
    'actor4': 'actor4_mojo',
    'actor5': 'actor5_mojo',
    'actor6': 'actor6_mojo',
    'producer1': 'producer1_mojo',
    'producer2': 'producer2_mojo',
    'producer3': 'producer3_mojo',
    'producer4': 'producer4_mojo',
    'producer5': 'producer5_mojo',
    'producer6': 'producer6_mojo',
    'cinematographer': 'cinematographer_mojo',
    'composer1': 'composer1_mojo',
    'composer2': 'composer2_mojo'
}, axis=1, inplace=True)

# Cast `release_year` as Nullable Integer Data Type to allow comparisons as integers and still have np.nan values
mojo['release_year'] = mojo['release_date_mojo'].dt.year.astype("Int64")

mojo['runtime_mojo'] = mojo['runtime_mojo'].astype("Int64")

# Reorder the columns 
mojo = mojo[['title', 'release_year', 'release_date_mojo', 'distributor_mojo', 'runtime_mojo',
             'rating_mojo', 'genres_mojo', 'production_budget_mojo', 'domestic_gross_mojo',
             'foreign_gross_mojo', 'worldwide_gross_mojo', 'adjusted_domestic_gross_2019_mojo', 'director1_mojo',
             'director2_mojo', 'writer1_mojo', 'writer2_mojo', 'writer3_mojo', 'actor1_mojo', 'actor2_mojo', 'actor3_mojo', 'actor4_mojo', 'actor5_mojo', 
             'actor6_mojo', 'producer1_mojo', 'producer2_mojo', 'producer3_mojo', 'producer4_mojo', 'producer5_mojo', 'producer6_mojo', 'cinematographer_mojo',
             'composer1_mojo', 'composer2_mojo']]

# Only analyze movies that were released up to 2018 to prevent incomplete box office numbers
mojo = mojo[~(mojo['release_year'] > 2018)]

# We only want entries that have a title
mojo = mojo[~mojo['title'].isna()]

# We will drop any duplicate rows
mojo.drop_duplicates(inplace=True)

In [4]:
mojo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18465 entries, 0 to 19228
Data columns (total 32 columns):
title                                18465 non-null object
release_year                         18075 non-null Int64
release_date_mojo                    18075 non-null datetime64[ns]
distributor_mojo                     18440 non-null object
runtime_mojo                         17763 non-null Int64
rating_mojo                          18465 non-null object
genres_mojo                          18465 non-null object
production_budget_mojo               3316 non-null object
domestic_gross_mojo                  16499 non-null float64
foreign_gross_mojo                   5766 non-null float64
worldwide_gross_mojo                 7650 non-null float64
adjusted_domestic_gross_2019_mojo    16435 non-null float64
director1_mojo                       6273 non-null object
director2_mojo                       215 non-null object
writer1_mojo                         4293 non-null object
wri

###  Fix `production-budget_mojo` column

We must properly convert `production_budget_mojo` into a numeric form.

Right now, it has entries like `$30 million` in it.

There are a couple bad entries in it (e.g. `PG` and `Unrated`) from a bad web-scrape that we will fix. Doing some research, the websites for the movie in question is `https://www.boxofficemojo.com/movies/?id=picnicathangingrock98.htm` and `https://www.boxofficemojo.com/movies/?id=zyzzyxroad.htm`.

In [5]:
mojo['production_budget_mojo'].value_counts(dropna=False)

NaN                  15149
$30 million            137
$20 million            127
$40 million            124
$25 million            113
$35 million            109
$50 million             86
$60 million             85
$15 million             85
$10 million             81
$12 million             74
$80 million             63
$5 million              60
$70 million             54
$100 million            53
$75 million             50
$6 million              49
$18 million             48
$45 million             47
$150 million            44
$3 million              43
$7 million              43
$8 million              42
$13 million             41
$65 million             40
$22 million             40
$17 million             40
$90 million             39
$55 million             39
$14 million             38
$16 million             35
$19 million             33
$28 million             32
$9 million              32
$2 million              32
$85 million             31
$11 million             31
$

#### Fix webscraping errors

Fix the `PG` entry.

In [6]:
mojo[mojo['production_budget_mojo'] == 'PG']

,title,release_year,release_date_mojo,distributor_mojo,runtime_mojo,rating_mojo,genres_mojo,production_budget_mojo,domestic_gross_mojo,foreign_gross_mojo,worldwide_gross_mojo,adjusted_domestic_gross_2019_mojo,director1_mojo,director2_mojo,writer1_mojo,writer2_mojo,writer3_mojo,actor1_mojo,actor2_mojo,actor3_mojo,actor4_mojo,actor5_mojo,actor6_mojo,producer1_mojo,producer2_mojo,producer3_mojo,producer4_mojo,producer5_mojo,producer6_mojo,cinematographer_mojo,composer1_mojo,composer2_mojo
1245,Picnic at Hanging Rock (Re-issue),NaN,NaT,Domestic Lifetime Adj. Gross: n/a,NaN,1 hrs. 47 min.,"June 26, 1998",PG,NaN,NaN,NaN,446100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# From the website
mojo.at[1245, 'release_year'] = 1998
mojo.at[1245, 'release_date_mojo'] = pd.to_datetime('19980626', format='%Y%m%d', errors='ignore')
mojo.at[1245, 'distributor_mojo'] = 'Kit Parker Films'
mojo.at[1245, 'runtime_mojo'] = 107
mojo.at[1245, 'rating_mojo'] = 'PG'
mojo.at[1245, 'genres_mojo'] = 'Unknown'
mojo.at[1245, 'production_budget_mojo'] = np.nan
mojo.at[1245, 'domestic_gross_mojo'] = 232201

In [8]:
mojo[mojo['title'] == 'Picnic at Hanging Rock (Re-issue)']

,title,release_year,release_date_mojo,distributor_mojo,runtime_mojo,rating_mojo,genres_mojo,production_budget_mojo,domestic_gross_mojo,foreign_gross_mojo,worldwide_gross_mojo,adjusted_domestic_gross_2019_mojo,director1_mojo,director2_mojo,writer1_mojo,writer2_mojo,writer3_mojo,actor1_mojo,actor2_mojo,actor3_mojo,actor4_mojo,actor5_mojo,actor6_mojo,producer1_mojo,producer2_mojo,producer3_mojo,producer4_mojo,producer5_mojo,producer6_mojo,cinematographer_mojo,composer1_mojo,composer2_mojo
1245,Picnic at Hanging Rock (Re-issue),1998,1998-06-26,Kit Parker Films,107,PG,Unknown,NaN,232201.0,NaN,NaN,446100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fix the `Unrated` entry.

In [9]:
mojo[mojo['production_budget_mojo'] == 'Unrated']

,title,release_year,release_date_mojo,distributor_mojo,runtime_mojo,rating_mojo,genres_mojo,production_budget_mojo,domestic_gross_mojo,foreign_gross_mojo,worldwide_gross_mojo,adjusted_domestic_gross_2019_mojo,director1_mojo,director2_mojo,writer1_mojo,writer2_mojo,writer3_mojo,actor1_mojo,actor2_mojo,actor3_mojo,actor4_mojo,actor5_mojo,actor6_mojo,producer1_mojo,producer2_mojo,producer3_mojo,producer4_mojo,producer5_mojo,producer6_mojo,cinematographer_mojo,composer1_mojo,composer2_mojo
2746,Zyzzyx Road,NaN,NaT,NaN,NaN,1 hrs. 30 min.,"February 25, 2006",Unrated,30.0,NaN,NaN,NaN,John Penney,NaN,NaN,NaN,NaN,Katherine Heigl,Tom Sizemore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# From the website
mojo.at[2746, 'release_year'] = 2006
mojo.at[2746, 'release_date_mojo'] = pd.to_datetime('20060225', format='%Y%m%d', errors='ignore')
mojo.at[2746, 'distributor_mojo'] = 'Regent Releasing'
mojo.at[2746, 'runtime_mojo'] = 90
mojo.at[2746, 'rating_mojo'] = 'Unrated'
mojo.at[2746, 'genres_mojo'] = 'Thriller'
mojo.at[2746, 'production_budget_mojo'] = '2000000'
mojo.at[2746, 'domestic_gross_mojo'] = 30

In [11]:
mojo[mojo['title'] == 'Zyzzyx Road']

,title,release_year,release_date_mojo,distributor_mojo,runtime_mojo,rating_mojo,genres_mojo,production_budget_mojo,domestic_gross_mojo,foreign_gross_mojo,worldwide_gross_mojo,adjusted_domestic_gross_2019_mojo,director1_mojo,director2_mojo,writer1_mojo,writer2_mojo,writer3_mojo,actor1_mojo,actor2_mojo,actor3_mojo,actor4_mojo,actor5_mojo,actor6_mojo,producer1_mojo,producer2_mojo,producer3_mojo,producer4_mojo,producer5_mojo,producer6_mojo,cinematographer_mojo,composer1_mojo,composer2_mojo
2746,Zyzzyx Road,2006,2006-02-25,Regent Releasing,90,Unrated,Thriller,2000000,30.0,NaN,NaN,NaN,John Penney,NaN,NaN,NaN,NaN,Katherine Heigl,Tom Sizemore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Fix numeric strings

Fix strings like `million` in `production_budget_mojo`. Also, convert its type to "Int64".

In [12]:
mojo['production_budget_mojo'].head(10)

0    $25.1 million
1              NaN
2              NaN
3              NaN
4              NaN
5              NaN
6              NaN
7              NaN
8              NaN
9              NaN
Name: production_budget_mojo, dtype: object

In [13]:
mojo['production_budget_mojo'] = mojo['production_budget_mojo'].str.replace('$', '').str.replace(',', '').str.strip().str.split(' ')

In [14]:
mojo['production_budget_mojo'].sort_values(ascending=False)

1067       [99, million]
13827      [98, million]
3912       [98, million]
15184      [97, million]
8968       [97, million]
10602           [950000]
12827           [950000]
16057      [95, million]
3129       [95, million]
6460       [95, million]
14575      [95, million]
12306      [95, million]
16939      [95, million]
13213      [95, million]
1110       [95, million]
4909       [95, million]
10039           [944149]
2129       [94, million]
11267      [94, million]
15279      [94, million]
15281      [94, million]
10545      [94, million]
4433       [93, million]
15277      [93, million]
13010      [93, million]
16021      [93, million]
6817       [92, million]
11508      [92, million]
7939       [92, million]
17148      [92, million]
9312       [92, million]
11643      [92, million]
15668           [900000]
1055       [90, million]
3266       [90, million]
14706      [90, million]
16287      [90, million]
2849       [90, million]
16564      [90, million]
10197      [90, million]


In [15]:
results = []

for entry in mojo['production_budget_mojo']:
    if type(entry) is list:
        if len(entry) > 1:
            budget = float(entry[0]) * 1000000
            budget = int(budget)
            results.append(budget)
        else:
            results.append(int(entry[0]))
    else:
        results.append(entry)
        
mojo['production_budget_mojo'] = results
mojo['production_budget_mojo'] = mojo['production_budget_mojo'].astype("Int64")

mojo['production_budget_mojo'].head()

0    25100000
1         NaN
2         NaN
3         NaN
4         NaN
Name: production_budget_mojo, dtype: Int64

In [16]:
mojo['production_budget_mojo'].sort_values(ascending=False)

5838     317000000
1300     300000000
5490     275000000
6231     270000000
10728    260000000
5690     258000000
7401     250000000
1303     250000000
11910    250000000
1963     250000000
9782     250000000
4304     250000000
9645     250000000
12752    250000000
5837     245000000
5668     245000000
9638     237000000
1302     230000000
9104     230000000
3409     225000000
1301     225000000
2943     225000000
13185    225000000
3103     220000000
11417    217000000
15203    215000000
18437    215000000
2671     210000000
11414    210000000
7429     209000000
17636    207000000
11215    200000000
13167    200000000
12818    200000000
5314     200000000
5689     200000000
17938    200000000
11416    200000000
11376    200000000
10869    200000000
16133    200000000
17339    200000000
17338    200000000
1586     200000000
16081    200000000
1891     200000000
2669     200000000
7090     200000000
7024     200000000
18558    200000000
10870    200000000
8953     200000000
11415    195

## Import The Numbers dataset and fix small issues

In [17]:
numbers = pd.read_csv("the_numbers_movie_data.csv", parse_dates=["release_date"])

# Cast `release_year` as Nullable Integer Data Type to allow comparisons as integers and still have np.nan values
numbers['release_year'] = numbers['release_date'].dt.year.astype("Int64")

numbers.rename(mapper={
    'release_date': 'release_date_numbers',
    'rank': 'rank_numbers',
    'production_budget': 'production_budget_numbers',
    'domestic_gross': 'domestic_gross_numbers',
    'worldwide_gross': 'worldwide_gross_numbers'
}, axis=1, inplace=True)

# Reorder columns
numbers = numbers[['title', 'release_year', 'release_date_numbers', 'rank_numbers', 'production_budget_numbers', 'domestic_gross_numbers', 'worldwide_gross_numbers']]

# Only analyze movies that were released up to 2018 to prevent incomplete box office numbers
numbers = numbers[~(numbers['release_year'] > 2018)]

In [18]:
numbers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5712 entries, 0 to 5781
Data columns (total 7 columns):
title                        5712 non-null object
release_year                 5712 non-null Int64
release_date_numbers         5712 non-null datetime64[ns]
rank_numbers                 5712 non-null int64
production_budget_numbers    5712 non-null int64
domestic_gross_numbers       5712 non-null int64
worldwide_gross_numbers      5712 non-null int64
dtypes: Int64(1), datetime64[ns](1), int64(4), object(1)
memory usage: 362.6+ KB


## Merge Two DataFrames

We will merge the two dataframes on the columns `title` and `release_year`.

We will search for duplicate titles that may exist in both datasets but happen to have different years of release.

We will combine the data from both datasets into one row of data for these instances.

In [19]:
outer_merged = pd.merge(mojo, numbers, how='outer', on=['title', 'release_year'], suffixes=['_mojo', '_numbers'])

In [20]:
outer_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20200 entries, 0 to 20199
Data columns (total 37 columns):
title                                20200 non-null object
release_year                         19812 non-null object
release_date_mojo                    18077 non-null datetime64[ns]
distributor_mojo                     18441 non-null object
runtime_mojo                         17765 non-null Int64
rating_mojo                          18465 non-null object
genres_mojo                          18465 non-null object
production_budget_mojo               3315 non-null Int64
domestic_gross_mojo                  16500 non-null float64
foreign_gross_mojo                   5766 non-null float64
worldwide_gross_mojo                 7650 non-null float64
adjusted_domestic_gross_2019_mojo    16435 non-null float64
director1_mojo                       6273 non-null object
director2_mojo                       215 non-null object
writer1_mojo                         4293 non-null object
wri

In [21]:
# We want to remove duplicates from the two datasets.
# We will create a subset of the outer_merged dataframe that contains duplicated titles.
# Then we will sort it in descending order based on `title` and `release_year`.
# If we find the same movie with consecutive release years, there's a good chance it's a duplicated entry.
# We will check if the movie has already been merged on by checking if it has data from both `mojo` and `numbers`.
# Then we will extract the duplicates that are from consecutive years.
duplicated_df = pd.DataFrame(columns=outer_merged.columns)
previous_row = None

# We loop through a subset of outer_merged that contains duplicated titles
for index, row in outer_merged[outer_merged['title'].duplicated(keep=False)].sort_values(['title', 'release_year'], ascending=[False, False]).iterrows():
    if previous_row is None:
        previous_row = outer_merged.iloc[index].copy()
        continue
        
    if row['title'] != previous_row['title']:
        previous_row = outer_merged.iloc[index].copy()
        continue
        
    if row['release_year'] == previous_row['release_year'] - 1:
        # If any title has both `release_date_mojo` and `release_date_numbers`, then it has already been merged on. 
        # We will assume another matching title in this instance is a separate movie. 
        if (pd.notnull(row['release_date_mojo']) & pd.notnull(row['release_date_numbers'])) | (pd.notnull(previous_row['release_date_mojo']) & pd.notnull(previous_row['release_date_numbers'])):
            previous_row = outer_merged.iloc[index].copy()
            continue
            
        else:
            row_copy = row.copy()
            duplicated_df = duplicated_df.append([previous_row, row_copy])
            previous_row = outer_merged.iloc[index].copy()
        
    else:
        previous_row = outer_merged.iloc[index].copy()

In [22]:
temp_df = pd.DataFrame(columns=outer_merged.columns)
previous_row = None
previous_index = None
indexes_to_delete = []

for index, row in duplicated_df.iterrows():
    if previous_row is None:
        previous_row = outer_merged.iloc[index].copy()
        previous_index = index
        continue
        
    if row['title'] != previous_row['title']:
        previous_row = outer_merged.iloc[index].copy()
        previous_index = index
        continue
        
    if row['release_year'] == previous_row['release_year'] - 1:
        the_numbers_check = pd.notnull(previous_row['production_budget_numbers'])
        
        if the_numbers_check:
            # The current row is populated from The Numbers database, so we must populate the previous_row with The Numbers data.
            # We will only keep the entry with the later release date, for consistency.
            for column in numbers.columns:
                row.loc[column] = previous_row.loc[column]
                
            indexes_to_delete.append(previous_index)
            indexes_to_delete.append(index)
            temp_df = temp_df.append(row)
        else:
            for column in mojo.columns:
                row.loc[column] = previous_row.loc[column]
            indexes_to_delete.append(previous_index)
            indexes_to_delete.append(index)
            temp_df = temp_df.append(row)
            
    previous_row = outer_merged.iloc[index].copy()
    previous_index = index
    
outer_merged.drop(labels=indexes_to_delete, axis=0, inplace=True)
outer_merged = outer_merged.append(temp_df)
outer_merged.sort_values(['title', 'release_year'], ascending=[True, True], inplace=True)
outer_merged.reset_index(drop=True, inplace=True)

In [23]:
outer_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20166 entries, 0 to 20165
Data columns (total 37 columns):
title                                20166 non-null object
release_year                         19778 non-null object
release_date_mojo                    18077 non-null datetime64[ns]
distributor_mojo                     18441 non-null object
runtime_mojo                         17765 non-null object
rating_mojo                          18465 non-null object
genres_mojo                          18465 non-null object
production_budget_mojo               3315 non-null object
domestic_gross_mojo                  16500 non-null float64
foreign_gross_mojo                   5766 non-null float64
worldwide_gross_mojo                 7650 non-null float64
adjusted_domestic_gross_2019_mojo    16435 non-null float64
director1_mojo                       6273 non-null object
director2_mojo                       215 non-null object
writer1_mojo                         4293 non-null object
w

## Consolidate Duplicate Columns

We now have multiple sources of similar information, such as `production budget`, `domestic gross`, `worldwide gross`, and `release date`.

We will combine these columns where appropriate to simplify our analysis.

### Consolidate `release_date_mojo` and `release_date_numbers`

We will use the release date from Box Office Mojo if it exists. If not, we will defer to The Numbers.

In [24]:
def select_release_date(row):
    if pd.notnull(row['release_date_mojo']):
        return row['release_date_mojo']
    elif pd.notnull(row['release_date_numbers']):
        return row['release_date_numbers']
    else:
        return np.nan

In [25]:
outer_merged['release_date'] = outer_merged.apply(select_release_date, axis=1)

### Consolidate `production_budget_mojo` and `production_budget_numbers`

Production budgets for movies are notoriously unreliable.

To illustrate this, we take the difference between the `production_budget_numbers` and `production_budget_mojo` columns. 

For the same movies, The Numbers reports aggregate production budgets that are \\$2.2 trillion less than Box Office Mojo's.

That equates to an average production budget difference of almost $1 million per movie.

In [26]:
production_budget_difference = outer_merged['production_budget_numbers'] - outer_merged['production_budget_mojo'] 
print("The sum of all production budget differences is", production_budget_difference.sum())
print("The average production budget difference is", production_budget_difference.mean())

The sum of all production budget differences is -2225965624.0
The average production budget difference is -972887.0734265734


To help account for these discrepancies, where both sites have a production budget, we will keep the average. Otherwise, we will keep whichever production budget data is there.

In [27]:
def select_production_budget(row):
    if pd.notnull(row['production_budget_mojo']) & pd.notnull(row['production_budget_numbers']):
        return round((row['production_budget_mojo'] + row['production_budget_numbers']) / 2)
    
    if pd.notnull(row['production_budget_mojo']):
        return row['production_budget_mojo']
    
    if pd.notnull(row['production_budget_numbers']):
        return row['production_budget_numbers']
    
    return np.nan

In [28]:
outer_merged['production_budget'] = outer_merged.apply(select_production_budget, axis=1)
outer_merged['production_budget'] = outer_merged['production_budget'].astype("Int64")

We now have production budget information for 6739 movies.

In [29]:
print("Box Office Mojo production budgets", outer_merged['production_budget_mojo'].notna().sum())
print("The Numbers production budgets", outer_merged['production_budget_numbers'].notna().sum())
print("Total useable production budgets", outer_merged['production_budget'].notna().sum())

Box Office Mojo production budgets 3315
The Numbers production budgets 5712
Total useable production budgets 6739


###  Consolidate `domestic_gross_mojo` and `domestic_gross_numbers`¶

Domestic box office gross is more reliable. 

For the same movies, The Numbers reports domestic grosses that are \\$120 million higher than Box Office Mojo's.

The per movie difference is only \\$30,600.

Like before, we will average where appropriate to smooth out the numbers.

In [30]:
domestic_gross_difference = outer_merged['domestic_gross_numbers'] - outer_merged['domestic_gross_mojo'] 
print("The sum of all production budget differences is", domestic_gross_difference.sum())
print("The average production budget difference is", domestic_gross_difference.mean())

The sum of all production budget differences is 120465195.0
The average production budget difference is 30621.554397559736


In [31]:
def select_domestic_gross(row):
    if pd.notnull(row['domestic_gross_mojo']) & pd.notnull(row['domestic_gross_numbers']):
        return round((row['domestic_gross_mojo'] + row['domestic_gross_numbers']) / 2)
    
    if pd.notnull(row['domestic_gross_mojo']):
        return row['domestic_gross_mojo']
    
    if pd.notnull(row['domestic_gross_numbers']):
        return row['domestic_gross_numbers']
    
    return np.nan

In [32]:
outer_merged['domestic_gross'] = outer_merged.apply(select_domestic_gross, axis=1)
outer_merged['domestic_gross'] = outer_merged['domestic_gross'].astype("Int64")

We now have domestic gross information for 18,278 movies.

In [33]:
print("Box Office Mojo domestic grosses", outer_merged['domestic_gross_mojo'].notna().sum())
print("The Numbers domestic grosses", outer_merged['domestic_gross_numbers'].notna().sum())
print("Total useable domestic grosses", outer_merged['domestic_gross'].notna().sum())

Box Office Mojo domestic grosses 16500
The Numbers domestic grosses 5712
Total useable domestic grosses 18278


### Consolidate `worldwide_gross_mojo` and `worldwide_gross_numbers`

Worldwide gross figures differ greatly.

For the same movies, The Numbers reports worldwide grosses that are \\$2 trillion less than Box Office Mojo's.

The per movie difference is $650,000.

Like before, we will average where appropriate to smooth out the numbers.

In [34]:
worldwide_gross_difference = outer_merged['worldwide_gross_numbers'] - outer_merged['worldwide_gross_mojo'] 
print("The sum of all worldwide gross differences is", worldwide_gross_difference.sum())
print("The average worldwide gross difference is", worldwide_gross_difference.mean())

The sum of all worldwide gross differences is -1974156300.0
The average worldwide gross difference is -652613.652892562


In [35]:
def select_worldwide_gross(row):
    if pd.notnull(row['worldwide_gross_mojo']) & pd.notnull(row['worldwide_gross_numbers']):
        return round((row['worldwide_gross_mojo'] + row['worldwide_gross_numbers']) / 2)
    
    if pd.notnull(row['worldwide_gross_mojo']):
        return row['worldwide_gross_mojo']
    
    if pd.notnull(row['worldwide_gross_numbers']):
        return row['worldwide_gross_numbers']
    
    return np.nan

In [36]:
outer_merged['worldwide_gross'] = outer_merged.apply(select_worldwide_gross, axis=1)
outer_merged['worldwide_gross'] = outer_merged['worldwide_gross'].astype("Int64")

We now have worldwide gross information for 10,337 movies.

In [37]:
print("Box Office Mojo worldwide grosses", outer_merged['worldwide_gross_mojo'].notna().sum())
print("The Numbers worldwide grosses", outer_merged['worldwide_gross_numbers'].notna().sum())
print("Total useable worldwide grosses", outer_merged['worldwide_gross'].notna().sum())

Box Office Mojo worldwide grosses 7650
The Numbers worldwide grosses 5712
Total useable worldwide grosses 10337


## Adjusting Ticket Prices For Inflation

We want to convert prices to 2018 dollars to normalize our prices. (We choose 2018 because we will ignore movies released in 2019 to avoid underrepresenting box office returns for movies that haven't had the time to earn their full potential. We don't want our data to be skewed lower because of that.) 

There are two ways to adjust box office grosses and production budgets for inflation. We could use the Consumer Price Index, or we could convert dollar amounts into contemporary ticket prices and calculate their equivalent in 2018 ticket prices. (For example, divide a 1940 movie's domestic gross by the 1940 ticket price, then calculate that amount by the 2018 ticket price.)

We will do both options and compare the results to determine which coversion seems more appropriate.

### Conversion using ticket prices 

We found historical ticket prices from [Box Office Mojo Adjusting Ticket Price For Inflation](https://www.boxofficemojo.com/about/adjuster.htm) and [National Association of Theatre Owners](https://www.natoonline.org/data/ticket-price/), but the datasets are missing years.

We create a custom function to fill in missing ticket prices in an incremental way. Filling in ticket prices using a forward fill or backwards fill seems less accurate, as movie ticket prices have consistently increased over time. (We would rather have a sliding scale than several discrete dollar amounts, given the overal trend in price appreciation.)

Our custom function should smooth out missing ticket prices nicely.

In [38]:
# Incomplete dictionary of ticket prices
ticket_prices = {
    "2019": 9.01,
    "2018": 9.11,
    "2017": 8.97,
    "2016": 8.65,
    "2015": 8.43,
    "2014": 8.17,
    "2013": 8.13,
    "2012": 7.96,
    "2011": 7.93,
    "2010": 7.89,
    "2009": 7.50,
    "2008": 7.18,
    "2007": 6.88,
    "2006": 6.55,
    "2005": 6.41,
    "2004": 6.21,
    "2003": 6.03,
    "2002": 5.81,
    "2001": 5.66,
    "2000": 5.39,
    "1999": 5.08,
    "1998": 4.69,    
    "1997": 4.59,
    "1996": 4.42,
    "1995": 4.35,
    "1994": 4.18,
    "1993": 4.14,
    "1992": 4.15,
    "1991": 4.21,
    "1990": 4.23,
    "1989": 3.97,
    "1988": 4.11,
    "1987": 3.91,
    "1986": 3.71,
    "1985": 3.55,
    "1984": 3.36,
    "1983": 3.15,
    "1982": 2.94,
    "1981": 2.78,
    "1980": 2.69,
    "1979": 2.51,
    "1978": 2.34,
    "1977": 2.23,
    "1976": 2.13,
    "1975": 2.05,
    "1974": 1.87,
    "1973": 1.77,
    "1972": 1.70,
    "1971": 1.65,
    "1970": 1.55,
    "1969": 1.42,
    "1968": 1.31,
    "1967": 1.20,
    "1966": 1.09,
    "1965": 1.01,
    "1964": 0.93,
    "1963": 0.85,
    "1962": 0.70,
    "1961": 0.69,
    "1959": 0.51,
    "1958": 0.68,
    "1956": 0.50,
    "1954": 0.45,
    "1953": 0.60,
    "1951": 0.53,
    "1949": 0.46,
    "1948": 0.40,
    "1945": 0.35,
    "1944": 0.32,
    "1943": 0.29,
    "1942": 0.27,
    "1941": 0.25,
    "1940": 0.24,
    "1939": 0.23,
    "1936": 0.25,
    "1935": 0.24,
    "1934": 0.23,
    "1929": 0.35,
    "1924": 0.25,
    "1910": 0.07    
}

# Custom function that loops through the dictionary keys.
# It sets a previous_closest_key, then loops until it finds a missing key, continually updating the previous_closest_key to be as close to the missing key as possible.
# When it finds a missing key, it then loops until it finds the next key in the dictionary (i.e. the next_closest_key).
# Then it calculates the distance from the next_closest_key and previous_closest_key as well as a price differential and a unit price per distance.
# It adds the missing key along with its calculated value to the dictionary.
# It then loops to fill in other missing keys until the next_closest_key.
# Then it resets previous_closest_key to the value of next_closest_key and repeats the process.
def fill_in_missing_prices(dictionary):
    dictionary_copy = dictionary.copy()
    keys_list = list(dictionary_copy.keys())
    keys_list.sort()
    
    previous_closest_key = None
    previous_closest_value = None

    next_closest_key = None
    next_closest_value = None

    current_missing_key = None
    current_missing_value = None

    for i in range(int(keys_list[0]), int(keys_list[-1])):
        # Check if we have set a previous key
        if previous_closest_key is None:
            if str(i) in dictionary_copy:
                previous_closest_key = str(i)
                previous_closest_value = dictionary_copy[previous_closest_key]
                continue

        if (str(i) in ticket_prices) & (current_missing_key is None):
            previous_closest_key = str(i)
            previous_closest_value = dictionary_copy[previous_closest_key]
            continue

        if str(i) not in ticket_prices:
            if current_missing_key is None:
                current_missing_key = str(i)
            else:
                continue

        if (str(i) in ticket_prices) & (current_missing_key is not None):
            # Set next_closest_key and next_closest_value
            next_closest_key = str(i)
            next_closest_value = dictionary_copy[next_closest_key]

            # Calculate distance from next_closest_key to previous_closest_key
            known_keys_distance = int(next_closest_key) - int(previous_closest_key)
            known_price_difference = next_closest_value - previous_closest_value
            unit_price_per_distance = known_price_difference / known_keys_distance

            # Calculate missing_key_value
            distance_from_previous_key = int(current_missing_key) - int(previous_closest_key)
            current_missing_value = previous_closest_value + distance_from_previous_key * unit_price_per_distance

            # Set current_missing_key and current_missing_value in the dictionary
            dictionary_copy[current_missing_key] = round(current_missing_value, 2)

            # Check if there are other in-between missing values to fill in before resetting all values
            if int(next_closest_key) - int(current_missing_key) > 1:
                for i in range(int(current_missing_key), int(next_closest_key)):
                    dictionary_copy[str(i)] = round(dictionary_copy[str(i - 1)] + unit_price_per_distance, 2)

            previous_closest_key = next_closest_key
            previous_closest_value = next_closest_value
            next_closest_key = None
            next_closest_value = None
            current_missing_key = None
            current_missing_value = None
            
    return dictionary_copy

historical_ticket_prices = fill_in_missing_prices(ticket_prices)

# We now have data for every year in our dataset and have smoothed out the values where missing.
pprint.pprint(historical_ticket_prices)

{'1910': 0.07,
 '1911': 0.08,
 '1912': 0.09,
 '1913': 0.1,
 '1914': 0.11,
 '1915': 0.12,
 '1916': 0.13,
 '1917': 0.14,
 '1918': 0.15,
 '1919': 0.16,
 '1920': 0.17,
 '1921': 0.18,
 '1922': 0.19,
 '1923': 0.2,
 '1924': 0.25,
 '1925': 0.27,
 '1926': 0.29,
 '1927': 0.31,
 '1928': 0.33,
 '1929': 0.35,
 '1930': 0.33,
 '1931': 0.31,
 '1932': 0.29,
 '1933': 0.27,
 '1934': 0.23,
 '1935': 0.24,
 '1936': 0.25,
 '1937': 0.24,
 '1938': 0.23,
 '1939': 0.23,
 '1940': 0.24,
 '1941': 0.25,
 '1942': 0.27,
 '1943': 0.29,
 '1944': 0.32,
 '1945': 0.35,
 '1946': 0.37,
 '1947': 0.39,
 '1948': 0.4,
 '1949': 0.46,
 '1950': 0.49,
 '1951': 0.53,
 '1952': 0.56,
 '1953': 0.6,
 '1954': 0.45,
 '1955': 0.47,
 '1956': 0.5,
 '1957': 0.59,
 '1958': 0.68,
 '1959': 0.51,
 '1960': 0.6,
 '1961': 0.69,
 '1962': 0.7,
 '1963': 0.85,
 '1964': 0.93,
 '1965': 1.01,
 '1966': 1.09,
 '1967': 1.2,
 '1968': 1.31,
 '1969': 1.42,
 '1970': 1.55,
 '1971': 1.65,
 '1972': 1.7,
 '1973': 1.77,
 '1974': 1.87,
 '1975': 2.05,
 '1976': 2.13,
 '19

In [39]:
# Function to convert price columns to 2018 prices
def apply_price_transform(dictionary, value):
    if pd.isnull(value):
        return np.nan
    else:
        return dictionary['2018'] / dictionary[str(value)]

In [40]:
# Create a price conversion series to apply to our numeric columns
outer_merged['price_conversion'] = outer_merged['release_year'].apply(lambda x: apply_price_transform(historical_ticket_prices, x))

outer_merged['price_conversion'].head()

0    1.080664
1    1.268802
2    1.144472
3    1.080664
4    1.000000
Name: price_conversion, dtype: float64

In [41]:
# Now let's convert all dollar-valued columns into their 2018-priced equivalents
# outer_merged['production_budget_mojo_2019'] = (outer_merged['production_budget_mojo'] * outer_merged['price_conversion']).astype("float64")
# outer_merged['production_budget_mojo_2019'] = round(outer_merged['production_budget_mojo_2019'])
# outer_merged['production_budget_mojo_2019'] = outer_merged['production_budget_mojo_2019'].astype("Int64")

# outer_merged['domestic_gross_mojo_2019'] = (outer_merged['domestic_gross_mojo'] * outer_merged['price_conversion']).astype("float64")
# outer_merged['domestic_gross_mojo_2019'] = round(outer_merged['domestic_gross_mojo_2019'])
# outer_merged['domestic_gross_mojo_2019'] = outer_merged['domestic_gross_mojo_2019'].astype("Int64")

# outer_merged['foreign_gross_mojo_2019'] = (outer_merged['foreign_gross_mojo'] * outer_merged['price_conversion']).astype("float64")
# outer_merged['foreign_gross_mojo_2019'] = round(outer_merged['foreign_gross_mojo_2019'])
# outer_merged['foreign_gross_mojo_2019'] = outer_merged['foreign_gross_mojo_2019'].astype("Int64")

# outer_merged['worldwide_gross_mojo_2019'] = (outer_merged['worldwide_gross_mojo'] * outer_merged['price_conversion']).astype("float64")
# outer_merged['worldwide_gross_mojo_2019'] = round(outer_merged['worldwide_gross_mojo_2019'])
# outer_merged['worldwide_gross_mojo_2019'] = outer_merged['worldwide_gross_mojo_2019'].astype("Int64")

# outer_merged['production_budget_numbers_2019'] = (outer_merged['production_budget_numbers'] * outer_merged['price_conversion']).astype("float64")
# outer_merged['production_budget_numbers_2019'] = round(outer_merged['production_budget_numbers_2019'])
# outer_merged['production_budget_numbers_2019'] = outer_merged['production_budget_numbers_2019'].astype("Int64")

# outer_merged['domestic_gross_numbers_2019'] = (outer_merged['domestic_gross_numbers'] * outer_merged['price_conversion']).astype("float64")
# outer_merged['domestic_gross_numbers_2019'] = round(outer_merged['domestic_gross_numbers_2019'])
# outer_merged['domestic_gross_numbers_2019'] = outer_merged['domestic_gross_numbers_2019'].astype("Int64")

# outer_merged['worldwide_gross_numbers_2019'] = (outer_merged['worldwide_gross_numbers'] * outer_merged['price_conversion']).astype("float64")
# outer_merged['worldwide_gross_numbers_2019'] = round(outer_merged['worldwide_gross_numbers_2019'])
# outer_merged['worldwide_gross_numbers_2019'] = outer_merged['worldwide_gross_numbers_2019'].astype("Int64")

outer_merged['production_budget_ticket_conversion_2018'] = (outer_merged['production_budget'] * outer_merged['price_conversion']).astype("float64")
outer_merged['production_budget_ticket_conversion_2018'] = round(outer_merged['production_budget_ticket_conversion_2018'])
outer_merged['production_budget_ticket_conversion_2018'] = outer_merged['production_budget_ticket_conversion_2018'].astype("Int64")

outer_merged['domestic_gross_ticket_conversion_2018'] = (outer_merged['domestic_gross'] * outer_merged['price_conversion']).astype("float64")
outer_merged['domestic_gross_ticket_conversion_2018'] = round(outer_merged['domestic_gross_ticket_conversion_2018'])
outer_merged['domestic_gross_ticket_conversion_2018'] = outer_merged['domestic_gross_ticket_conversion_2018'].astype("Int64")

outer_merged['worldwide_gross_ticket_conversion_2018'] = (outer_merged['worldwide_gross'] * outer_merged['price_conversion']).astype("float64")
outer_merged['worldwide_gross_ticket_conversion_2018'] = round(outer_merged['worldwide_gross_ticket_conversion_2018'])
outer_merged['worldwide_gross_ticket_conversion_2018'] = outer_merged['worldwide_gross_ticket_conversion_2018'].astype("Int64")

# Reset index 
outer_merged.reset_index(drop=True, inplace=True)

In [42]:
outer_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20166 entries, 0 to 20165
Data columns (total 45 columns):
title                                       20166 non-null object
release_year                                19778 non-null object
release_date_mojo                           18077 non-null datetime64[ns]
distributor_mojo                            18441 non-null object
runtime_mojo                                17765 non-null object
rating_mojo                                 18465 non-null object
genres_mojo                                 18465 non-null object
production_budget_mojo                      3315 non-null object
domestic_gross_mojo                         16500 non-null float64
foreign_gross_mojo                          5766 non-null float64
worldwide_gross_mojo                        7650 non-null float64
adjusted_domestic_gross_2019_mojo           16435 non-null float64
director1_mojo                              6273 non-null object
director2_mojo           

### Conversion using CPI (Consumer Price Index)

`cpi` is a python package put out by the [Los Angeles Times Data Desk](https://github.com/datadesk).

With it, we can convert the dollar amounts from any year to 2018 dollars. Let's do this and compare the results with the ticket price conversion.

The `cpi` conversion won't work with null release years. Let's examine how many null entries we have there. 

In [43]:
outer_merged['release_date'].value_counts(dropna=False)

NaT           388
2018-09-14     26
2013-09-06     26
2016-10-21     26
2017-10-20     25
2015-12-31     24
2014-12-31     24
2016-03-11     24
2016-04-15     24
2014-10-17     24
2016-10-07     23
2013-10-04     23
2017-12-01     23
2014-09-12     23
2018-09-28     22
2007-10-19     22
2018-05-11     22
2011-09-09     22
2015-10-16     22
2018-04-13     22
2017-08-11     22
2015-06-26     22
2017-04-07     22
2012-09-21     22
2017-09-22     22
2018-03-23     22
2018-10-26     21
2015-09-18     21
2013-09-20     21
2012-10-12     21
2008-10-10     21
2016-08-26     21
2014-09-19     21
2014-04-04     21
2017-09-08     21
2008-10-24     21
2016-08-12     21
2017-10-06     20
2016-03-18     20
2005-10-21     20
2007-03-23     20
2017-10-13     20
2016-04-01     20
2017-09-29     20
2012-04-27     20
2015-12-04     20
2016-09-09     20
2018-10-19     20
2006-03-17     20
2012-12-07     20
2017-10-27     20
2011-09-30     20
2013-03-01     20
2016-09-30     20
2013-09-13     20
2015-04-17

We are only missing 388 entries from `release_date`. Let's remove these entries.

In [44]:
outer_merged = outer_merged[outer_merged['release_date'].notna()]

In [45]:
outer_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19778 entries, 0 to 20165
Data columns (total 45 columns):
title                                       19778 non-null object
release_year                                19778 non-null object
release_date_mojo                           18077 non-null datetime64[ns]
distributor_mojo                            18077 non-null object
runtime_mojo                                17608 non-null object
rating_mojo                                 18077 non-null object
genres_mojo                                 18077 non-null object
production_budget_mojo                      3305 non-null object
domestic_gross_mojo                         16486 non-null float64
foreign_gross_mojo                          5693 non-null float64
worldwide_gross_mojo                        7572 non-null float64
adjusted_domestic_gross_2019_mojo           16434 non-null float64
director1_mojo                              6204 non-null object
director2_mojo           

In [46]:
outer_merged['domestic_cpi_adjusted'] = outer_merged.apply(lambda x: cpi.inflate(x['domestic_gross'], x['release_year']), axis=1)
outer_merged['domestic_cpi_adjusted'] = round(outer_merged['domestic_cpi_adjusted'])
outer_merged['domestic_cpi_adjusted'] = outer_merged['domestic_cpi_adjusted'].astype("Int64")

outer_merged['worldwide_cpi_adjusted'] = outer_merged.apply(lambda x: cpi.inflate(x['worldwide_gross'], x['release_year']), axis=1)
outer_merged['worldwide_cpi_adjusted'] = round(outer_merged['worldwide_cpi_adjusted'])
outer_merged['worldwide_cpi_adjusted'] = outer_merged['worldwide_cpi_adjusted'].astype("Int64")

outer_merged['production_budget_cpi_adjusted'] = outer_merged.apply(lambda x: cpi.inflate(x['production_budget'], x['release_year']), axis=1)
outer_merged['production_budget_cpi_adjusted'] = round(outer_merged['production_budget_cpi_adjusted'])
outer_merged['production_budget_cpi_adjusted'] = outer_merged['production_budget_cpi_adjusted'].astype("Int64")

### Compare two methods and choose winner

In [47]:
outer_merged['domestic_conversion_difference'] = outer_merged['domestic_gross_ticket_conversion_2018'] - outer_merged['domestic_cpi_adjusted']

In [48]:
outer_merged[['title', 'domestic_cpi_adjusted', 'domestic_gross_ticket_conversion_2018', 'domestic_conversion_difference']].sort_values('domestic_conversion_difference', ascending=False)

,title,domestic_cpi_adjusted,domestic_gross_ticket_conversion_2018,domestic_conversion_difference
6237,Gone with the Wind,3608832557,7912491451,4303658894
13651,Snow White and the Seven Dwarfs,3224728056,7019463239,3794735183
1726,Bambi,1579387159,3459170753,1879783594
11713,Pinocchio,1511611838,3199017647,1687405809
5233,Fantasia,1364246361,2910245909,1545999548
1727,Bambi (Re-issue) (1947),1151344175,2388388555,1237044380
8632,Lady and the Tramp,877022361,1814291893,937269532
3346,Cinderella (1950),970472801,1731665035,761192234
32,101 Dalmatians,1216735307,1912836127,696100820
3344,Cinderella,885647095,1580306122,694659027


In [49]:
print("The sum of all domestic conversion differences is", outer_merged['domestic_conversion_difference'].sum())
print("The average domestic conversion difference is", outer_merged['domestic_conversion_difference'].mean())

The sum of all domestic conversion differences is 89337124571
The average domestic conversion difference is 4891432.576160753


It seems like using the CPI would give us less extreme dollar values. The average difference in dollar amount conversions per movie is about $5 million.

In aggregate, using the ticket price conversion as opposed to CPI conversion gives us a total amount that is almost $90 billion higher.

Since we want to smooth out our dataset and avoid such extreme values, we are leaning towards using the CPI.

In [50]:
outer_merged['worldwide_conversion_difference'] = outer_merged['worldwide_gross_ticket_conversion_2018'] - outer_merged['worldwide_cpi_adjusted']
outer_merged[['title', 'worldwide_cpi_adjusted', 'worldwide_gross_ticket_conversion_2018', 'worldwide_conversion_difference']].sort_values('worldwide_conversion_difference', ascending=False)

,title,worldwide_cpi_adjusted,worldwide_gross_ticket_conversion_2018,worldwide_conversion_difference
6237,Gone with the Wind,7161768298,15702427180,8540658882
1726,Bambi,4124372009,9033191734,4908819725
13651,Snow White and the Seven Dwarfs,3224728056,7019463239,3794735183
3344,Cinderella,2746458483,4900648552,2154190069
11713,Pinocchio,1512022864,3199887500,1687864636
5233,Fantasia,1423281309,3036180800,1612899491
32,101 Dalmatians,1813009454,2850241924,1037232470
18446,Titanic,3438579733,4362154141,923574408
17780,The Ten Commandments,788720772,1556616882,767896110
2004,Ben-Hur,632159862,1308607147,676447285


In [51]:
print("The sum of all worldwide conversion differences is", outer_merged['worldwide_conversion_difference'].sum())
print("The average worldwide conversion difference is", outer_merged['worldwide_conversion_difference'].mean())

The sum of all worldwide conversion differences is 122687823622
The average worldwide conversion difference is 11959043.14475095


The average conversion difference in worldwide box office amounts is about $12 million.

The sum of all worldwide conversion differences is about $122 billion dollars.

In [52]:
outer_merged['production_budget_conversion_difference'] = outer_merged['production_budget_ticket_conversion_2018'] - outer_merged['production_budget_cpi_adjusted']
outer_merged[['title', 'production_budget_cpi_adjusted', 'production_budget_ticket_conversion_2018', 'production_budget_conversion_difference']].sort_values('production_budget_conversion_difference', ascending=False)

,title,production_budget_cpi_adjusted,production_budget_ticket_conversion_2018,production_budget_conversion_difference
2004,Ben-Hur,129436598,267941176,138504578
17780,The Ten Commandments,124630312,245970000,121339688
3436,Cleopatra (1963),361068889,471576471,110507582
3434,Cleopatra,344656667,450141176,105484509
10419,Mutiny on the Bounty (1962),157981225,247271429,89290204
10417,Mutiny on The Bounty,157981225,247271429,89290204
18446,Titanic,312905919,396949891,84043972
6237,Gone with the Wind,70454482,154473913,84019431
4684,Duel in the Sun,90140974,172351351,82210377
13835,Spartacus,101800135,182200000,80399865


In [53]:
print("The sum of all production budget conversion differences is", outer_merged['production_budget_conversion_difference'].sum())
print("The average production budget conversion difference is", outer_merged['production_budget_conversion_difference'].mean())

The sum of all production budget conversion differences is 31543675489
The average production budget conversion difference is 4687721.130777233


The average conversion difference in production budget is about $4.7 million.

The sum of all production budget differences is about $31.5 billion.

### Conversion choice

There are many ways to calculate price inflation. In this study, because it results in less extreme price differences over time, we will use the CPI method.

For example, in looking at Gone With The Wind with the CPI method, the domestic gross in 2018 dollars is \\$3.6 billion dollars. Using the ticket price conversion method, we get a domestic gross in 2018 dollars of \\$7.9 billion.

Gone With The Wind was by far the biggest movie of its time, but it seems more sensible to compare its success to a current dollar amount of $3.6 billion instead of more than double that amount.

## Creating `release_week` column

We will be analyzing box office performance in our analysis. One of the data points we will use is the release week of a movie, scaled from 1 - 53 based on how many weeks are in the year.

In [54]:
outer_merged['release_week'] = outer_merged['release_date'].dt.week.astype("Int64")

## Rename columns

We will rename some columns to make them easier to use in analysis.

In [55]:
outer_merged.rename(mapper={
    'production_budget': 'budget',
    'production_budget_cpi_adjusted': 'budget_adj',
    'domestic_gross': 'domestic',
    'domestic_cpi_adjusted': 'domestic_adj',
    'worldwide_gross': 'worldwide',
    'worldwide_cpi_adjusted': 'worldwide_adj'
}, axis=1, inplace=True)

## Clean up datatypes

In [56]:
# Clean up datatypes before saving data
outer_merged['release_week'] = outer_merged['release_week'].astype("Int64")
outer_merged['release_year'] = outer_merged['release_year'].astype("Int64")
outer_merged['runtime_mojo'] = outer_merged['runtime_mojo'].astype("Int64")
outer_merged['production_budget_mojo'] = outer_merged['production_budget_mojo'].astype("Int64")
outer_merged['domestic_gross_mojo'] = outer_merged['domestic_gross_mojo'].astype("Int64")
outer_merged['foreign_gross_mojo'] = outer_merged['foreign_gross_mojo'].astype("Int64")
outer_merged['worldwide_gross_mojo'] = outer_merged['worldwide_gross_mojo'].astype("Int64")
outer_merged['adjusted_domestic_gross_2019_mojo'] = outer_merged['adjusted_domestic_gross_2019_mojo'].astype("Int64")
outer_merged['rank_numbers'] = outer_merged['rank_numbers'].astype("Int64")
outer_merged['production_budget_numbers'] = outer_merged['production_budget_numbers'].astype("Int64")
outer_merged['domestic_gross_numbers'] = outer_merged['domestic_gross_numbers'].astype("Int64")
outer_merged['worldwide_gross_numbers'] = outer_merged['worldwide_gross_numbers'].astype("Int64")
outer_merged['budget'] = outer_merged['budget'].astype("Int64")
outer_merged['budget_adj'] = outer_merged['budget_adj'].astype("Int64")
outer_merged['domestic'] = outer_merged['domestic'].astype("Int64")
outer_merged['domestic_adj'] = outer_merged['domestic_adj'].astype("Int64")
outer_merged['worldwide'] = outer_merged['worldwide'].astype("Int64")
outer_merged['worldwide_adj'] = outer_merged['worldwide_adj'].astype("Int64")

In [57]:
outer_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19778 entries, 0 to 20165
Data columns (total 52 columns):
title                                       19778 non-null object
release_year                                19778 non-null Int64
release_date_mojo                           18077 non-null datetime64[ns]
distributor_mojo                            18077 non-null object
runtime_mojo                                17608 non-null Int64
rating_mojo                                 18077 non-null object
genres_mojo                                 18077 non-null object
production_budget_mojo                      3305 non-null Int64
domestic_gross_mojo                         16486 non-null Int64
foreign_gross_mojo                          5693 non-null Int64
worldwide_gross_mojo                        7572 non-null Int64
adjusted_domestic_gross_2019_mojo           16434 non-null Int64
director1_mojo                              6204 non-null object
director2_mojo                      

## Data Cleaning Summary

In our data cleaning we have accomplished the following tasks:
- Imported two different movie datasets from Box Office Mojo and The Numbers.
- Fixed errors from webscraping the data, including data in the wrong columns and converting string data into properly formatted numeric data.
- Merged our two datasets on the columns `title` and `release_year`.
- Eliminated duplicated movie data stemming from the same movie titles existing in multiple rows.
- Consolidated the `release_date_mojo` and `release_date_numbers` columns to maximize our useable release date information
- Consolidated several columns into `production_budget`, `domestic_gross`, and `worldwide_gross` columns by averaging duplicate data or selecting the appropriate column where needed.
- Tested two conversion methods and settled on using the CPI to convert our dollar amounts to 2018 dollars
- Created a `release_week` column
- Renamed columns to shorten them
- Cleaned up our datatypes to make our numeric data Int64

Now we can save our cleaned data and use that for our analysis.

In [58]:
# Save cleaned movie data as a pickle (commenting out to avoid overwriting later)
# outer_merged.to_pickle("cleaned_movie_data.pkl")

# Save cleaned movie data as a CSV (commenting out to avoid overwriting later)
# outer_merged.to_csv("cleaned_movie_data.csv", index=False)